<a href="https://colab.research.google.com/github/JunruL/Variational-Autoencoder/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fetch the Data

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!kaggle datasets download -d playlist/mnistzip
!unzip mnistzip.zip

### Import Libraries

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import cv2 as cv
import numpy as np
from torch import nn

### Creat a Custom Dataset

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data_lst: list, label_lst: list):
        self.data_lst = data_lst
        self.label_lst = label_lst

    def __len__(self):
        return len(self.data_lst)

    def __getitem__(self, idx):
        image = cv.imread(data_lst[idx], 0)
        image = np.expand_dims(image, axis=2)
        image = ToTensor()(image)
        label = label_lst[idx]
        return image, label

In [ ]:
data_lst = []
label_lst = []
for number in range(10):
  number_lst = os.listdir('./mnist_png/train/' + str(number))
  data_lst += ['./mnist_png/train/' + str(number) + '/' + str(file) for file in number_lst]
  label_lst += [number for _ in range(len(number_lst))]

In [ ]:
dataset = CustomImageDataset(number_lst, label_lst)
idx = 10000
image, label = dataset[idx]
print(label)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
for data in train_dataloader:
  sample_data, sample_label = data
  print(sample_data.shape)
  print(sample_label.shape)

### Build Neural Networks

Inference Model (Encoder / recognition Model): 

$$q_{\phi} (z | x)$$

We use a multivariate normal distribution for it: 

$$q_{\phi} (z | x) = \mathcal{N} (\vec{z}; \vec{\mu}, \text{diag}(\vec{\sigma}))$$

To make the the encoder differentiable, we can use the reparameterization trick: 

$$\vec{\epsilon} \sim \mathcal{N} (0, 1)$$

$$(\vec{\mu}, \log \vec{\sigma}) = \text{EncoderNeuralNet}_{\phi} (\vec{x})$$

$$\vec{z} = \vec{\mu} + \vec{\sigma} \odot \vec{\epsilon} $$

, where $\odot$ is the element-wise product.

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28*28, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.mean_layer = nn.Linear(128, 128)
        self.log_sd_layer = nn.Linear(128, 128)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.relu(x)
        mean = self.mean_layer(x)
        sd = torch.exp(self.log_sd_layer(x))

        # random vector from Normal distribution
        epsilon = torch.randn_like(sd)
        z = mean + sd * epsilon
        return z

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()   
        self.layer1 = nn.Linear(128, 256)
        self.layer2 = nn.Linear(256, 512)
        self.layer3 = nn.Linear(512, 28*28)
        self.relu = nn.ReLU()

    def forward(self, z):
        z = self.layer1(z)
        z = self.relu(z)
        z = self.layer2(z)
        z = self.relu(z)
        z = self.layer3(z)
        z = self.relu(z)

        z = torch.reshape(z, (z.shape[0], 1, 28, 28))
        return z

In [ ]:
for data in train_dataloader:
  sample_data, sample_label = data
  break

torch.Size([32, 784])


In [ ]:
encoder = Encoder()
z = encoder(sample_data)
z.shape

torch.Size([32, 128])

In [ ]:
decoder = Decoder()
x = decoder(z)
x.shape

torch.Size([32, 1, 28, 28])